# CNN

## 例題
MNISTデータセット（手書き数字文字画像）をCNNで分類するプログラムを作成せよ．

### STEP1：インポート
下記ライブラリをインポートする．

- torch
- torch.nn
- torch.nn.functional
- torch.optim
- torchvision
- torchvision.transforms

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

### STEP2：ネットワーク定義
ネットワークを定義するクラスを記述する． 

- クラスはnn.Moduleを継承する
- クラスは```__init__()```と```forward()```というメソッドを定義する
    - ```__init__()```：ネットワークの構造を定義
    - ```forward()```：ネットワークの順伝播を定義
- 2次元畳み込み層は```nn.Conv2d```クラスを用いる
- 全結合層は```nn.Linear```クラスを用いる
- 2次元最大値プーリング層は```F.max_pool2d()```を用いる
- 活性化関数ReLUは```F.relu()```を用いる

In [ ]:
# 任意のクラス名（ここではCNN）をつける
class CNN(nn.Module):

    # ネットワークの構造を定義
    def __init__(self):
        super(CNN, self).__init__()

        # 畳み込み層1の定義
        # 引数1：入力チャンネル数 = 1（MNISTはグレースケール画像）
        # 引数2：出力（畳み込み後の）チャンネル数 = 100
        # 引数3：カーネルサイズ = 3
        # 引数4：ストライド = 1
        self.conv1 = nn.Conv2d(1, 100, 3, 1)

        # 畳み込み層2の定義
        # 引数1：入力チャンネル数 = 100 (畳み込み層1の出力チャンネル数と同じ)
        # 引数2：出力（畳み込み後の）チャンネル数 = 30
        # 引数3：カーネルサイズ = 3
        # 引数4：ストライド = 1
        self.conv2 = nn.Conv2d(100, 30, 3, 1)

        # 全結合層1の定義
        # 引数1：入力サイズ = 750
        # 引数2：出力サイズ = 10 （画像のラベル数 0〜9）
        # 前段（MaxPooling層）の出力特徴マップが5x5，30チャンネルなので
        # 全結合層への入力は5x5x30=750になる
        self.fc1 = nn.Linear(750, 10)


    # ネットワークの順伝播を定義
    def forward(self, x):

        # 畳み込み層1の順伝播＋活性化関数ReLU
        # MNISTの画像サイズが28x28．カーネルサイズが3，ストライドが1なので，
        # 出力特徴マップのサイズは26x26になる
        x = self.conv1(x)
        x = F.relu(x)

        # MaxPooling
        # 入力特徴マップが26x26，カーネルサイズが2，ストライドが2なので
        # 出力特徴マップのサイズは13x13になる
        x = F.max_pool2d(x, 2, stride=2)

        # 畳み込み層2の順伝播＋活性化関数ReLU
        # 入力特徴マップが13x13．カーネルサイズが3，ストライドが1なので，
        # 出力特徴マップのサイズは10x10になる
        x = self.conv2(x)
        x = F.relu(x)

        # MaxPooling
        # 入力特徴マップが10x10，カーネルサイズが2，ストライドが2なので
        # 出力特徴マップのサイズは5x5になる
        x = F.max_pool2d(x, 2, stride=2)

        # 全結合層へ入力するために，特徴マップを1列へ整形する
        x = torch.flatten(x, 1)

        # 全結合層1の順伝播
        x = self.fc1(x)

        return x

In [ ]:
# 上記CNNクラスのインスタンスを生成
cnn = CNN()

# 実行するデバイスにインスタンスを移動
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn.to(device)

### STEP3：誤差関数・最適化器
誤差関数と最適化器を定義する．
- 誤差関数：```nn.MSELoss```と```nn.CrossEntropyLoss```がよく使われる
    - ```nn.MSELoss```：回帰問題でよく使う
    - ```nn.CrossEntropyLoss```：分類問題でよく使う（今回はこちらを使う）
- 最適化器：```optim.SGD```と```optim.Adam```がよく使われる
    - ```optim.SGD```：確率的勾配降下法（今回はこちらを使う）
    - ```optim.Adam```：Adam

In [ ]:
# 誤差関数（クロスエントロピー誤差）の定義
criterion = nn.CrossEntropyLoss()

# 最適化器（確率的勾配降下法）の定義
# 引数1：最適化するネットワークのパラメータ parameters()で取得可能
# 引数2：学習率 = 0.01
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

### STEP4：データセット
データセットを用意する．

- MNISTデータセット（28x28ピクセルの画像）は```torchvision```から読み出す
    - Training用データ（60000枚）
    - Test用データ（10000枚）
- バッチサイズ（一度にネットワークに入力するデータ数）を指定する
    - 大→データセット全てを入力するまでにかかる繰り返し回数が減るため学習時間が短くなるが メモリの消費が大きくなる
    - 小→データセット全てを入力するまでにかかる繰り返し回数が増えるため学習時間が長くなるが メモリの消費が小さくなる

In [ ]:
# ローカル環境で動作させる場合は
#   torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# でデータセットを./dataフォルダにダウンロードできるが，Colabだとダウンロードエラーが生じることがある． 
# 代わりにデータセットを直接ダウンロードし，ダウンロードした場所を指定する対策をとっている．

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [ ]:
# 画像整形の設定
transform_mnist = transforms.Compose([
    # PyTorchではTensor型と呼ばれる配列でデータを取り扱う
    # ここではダウンロードしてきた画像をTensor型に変換する
    transforms.ToTensor(),

    # ダウンロードした画像の値域は[0, 1]だが これを[-1, 1]に整形する
    # 入力画像inputsは下記の式に基づいて変換される
    # inputs[channel] = (inputs[channel] - mean[channel]) / std[channel]
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
    ])

# データセットの読み込み
# 引数1：データセットのパス
# 引数2：Training用かTest用か
# 引数3：ダウンロードしてくるか否か
# 引数4：画像整形の設定
trainset_mnist = torchvision.datasets.MNIST(root='./', train=True, download=False, transform=transform_mnist)
testset_mnist = torchvision.datasets.MNIST(root='./', train=False, download=False, transform=transform_mnist)

# ミニバッチの設定
# 引数1：データセット
# 引数2：バッチサイズ
# 引数3：データをシャッフルするか否か
# Trainingはデータの順番に依存するため 偏りが生じないようにシャッフルすることがある
# Testはデータの順番に依存しないため シャッフルは不要
batchsize = 100
trainloader_mnist = torch.utils.data.DataLoader(trainset_mnist, batch_size=batchsize, shuffle=True)
testloader_mnist = torch.utils.data.DataLoader(testset_mnist, batch_size=batchsize, shuffle=False)

### STEP5：ネットワーク学習
学習を下記のステップで行う．
- 入力データ・教師データをネットワークに入力
- ネットワークパラメータの微分値をリセット：```optimizer.zero_grad()```
- ネットワークの出力と教師信号の誤差を計算
- 誤差逆伝播：```loss.backward()```
- ネットワークパラメータの更新：```optimizer.step()```
- 以上を繰り返す（適宜ネットワークを評価）

In [ ]:
# エポック数：データセットを何周するか
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0

    # for文でTraining用データセットをループする
    for i, data in enumerate(trainloader_mnist, 0):
        # inputs：入力データ
        # labels：教師データ
        inputs, labels = data

        # 実行するデバイスにデータを移動
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 入力データを順伝播し出力を確認
        outputs = cnn(inputs)

        # ネットワークパラメータの微分値をリセット
        optimizer.zero_grad()

        # ネットワークの出力と教師信号の誤差を計算
        loss = criterion(outputs, labels)

        # 誤差逆伝播
        loss.backward()

        # ネットワークパラメータの更新
        optimizer.step()

        # 誤差のモニタリング
        # 100回に1回
        running_loss += loss.item()
        if i % 100 == 99:
            print("[{}, {}] train loss: {}".format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


    # ネットワークのTest
    test_loss = 0.0
    correct = 0

    # for文でTest用データセットをループする
    for i, data in enumerate(testloader_mnist, 0):
        # 入力データを順伝播しの出力を確認
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = cnn(inputs)

        # ネットワークの出力と教師信号の誤差を計算
        test_loss += criterion(outputs, labels).item()

        # ネットワークの出力ラベルを確認
        # ネットワークの出力のうちどこが最も大きいかを調べる
        pred = outputs.argmax(dim=1, keepdim=True)

        # ネットワークの出力ラベルと教師データが一致する数を数える
        correct += pred.eq(labels.view_as(pred)).sum().item()

    print("test loss: {}".format(epoch + 1, test_loss / len(testset_mnist)))
    print("test accuracy: {}".format(correct / len(testset_mnist)))

## 演習
CIFAR10データセット（物体カラー画像）をCNNで分類するプログラムを作成せよ．

### STEP1：インポート

### STEP2：ネットワーク定義

Note
- 例題とネットワーク構成を変える
- CIFAR10の画像サイズは32x32ピクセル
- CIFAR10はカラー画像なので3ch
- CIFAR10のラベル数は10

### STEP3：誤差関数・最適化器

### STEP4：データセット


In [ ]:
transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar10)
testset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar10)

batchsize = 100
trainloader_cifar10 = torch.utils.data.DataLoader(trainset_cifar10, batch_size=batchsize, shuffle=True)
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10, batch_size=batchsize, shuffle=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dataiter = iter(trainloader_cifar10)
images, labels = dataiter.next()

img = torchvision.utils.make_grid(images, nrow=10)
img = img / 2 + 0.5

npimg = img.numpy()
npimg = np.transpose(npimg, (1, 2, 0))
plt.imshow(npimg)
plt.show()

### STEP5：ネットワーク学習